In [8]:
from peft import LoraConfig, TaskType
import torch

peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

In [ ]:
from model import GPTConfig, GPT
import torch

ckpt_file = "evaluation/eval_models/random_8layers_ckpt_few_iters.pth"
n_layer = 8
n_head = 8
n_embd = 512
block_size = 1023
bias = False
dropout = 0.0


model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,bias=bias, vocab_size=None, dropout=dropout) # start with model_args from command line
checkpoint = torch.load(ckpt_file, map_location = 'cpu')
checkpoint_model_args = checkpoint['model_args']
# force these config attributes to be equal otherwise we can't even resume training
# the rest of the attributes (e.g. dropout) can stay as desired from command line
for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
    model_args[k] = checkpoint_model_args[k]
# create the model
gptconf = GPTConfig(**model_args)
model = GPT(gptconf)
state_dict = checkpoint['model']
# fix the keys of the state dictionary :(
# honestly no idea how checkpoints sometimes get this prefix, have to debug more
unwanted_prefix = '_orig_mod.'
iter_num = checkpoint['iter_num']
try:
    train_loss_list = checkpoint['train_loss_list']
except Exception as e:
    print('no train loss list found defaulting to empty')
try:
    val_loss_list = checkpoint['val_loss_list']
except Exception as e:
    print('no val loss list found defaulting to empty')
flag = False
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        flag = True
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)
best_val_loss = checkpoint['best_val_loss']
if flag:
    print('we did in face remove unwanted prefix')




/tmp/ipykernel_66480/1991922251.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_file, map_location = 'cpu')


number of parameters: 25.19M
we did in face remove unwanted prefix


In [ ]:
from transformers import AutoModelForSeq2SeqLM



In [7]:
from peft import get_peft_model

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

TypeError: 'GPTConfig' object is not subscriptable